下面，我们基于gnt-H的10层模型，应用上左右随机30度以内的旋转，再加上gamma随机校正的训练数据，进行训练。以观察精度增长情况。

这个应该很耗时。但是，希望在4万步时，精度能超过83%。（8层在4万步时，精度达到了84%）


In [ ]:
import tensorflow as tf
import numpy as np
import os
import random
from numpy import array

from skimage.io import imread, imsave
from skimage.color import rgb2gray
from skimage.transform import resize
from skimage.exposure import adjust_gamma
from skimage.transform import rotate

IMAGE_SIZE = 64
FULL_CHARSET_SIZE = 3755
CHARSET_SIZE = 37

def getRndGamma():
    rnd1 = np.random.random_sample()
    rnd2 = np.random.random_sample()+0.001
    rnd3 = np.random.random_sample()*100

    if (rnd1 > 0.5):
        return rnd3
    else:
        return rnd2
    
def input(dataset):
    return dataset.images, dataset.labels

class DataSetLoader:
    def __init__(self, data_dir):
        # Set CHARSET_SIZE to a small value if available computation power is limited.
        truncate_path = data_dir + ('%05d' % CHARSET_SIZE)
        print('Now processing path: ', truncate_path)
        image_names = []
        for root, sub_folder, file_list in os.walk(data_dir):
            if root < truncate_path:
                image_names += [os.path.join(root, file_path) for file_path in file_list]
        random.shuffle(image_names)
        self.labels = [int(file_name[len(data_dir):].split(os.sep)[0]) for file_name in image_names]
        images_rgb = [imread(file_name) for file_name in image_names]
        image_resized = [resize(image, (IMAGE_SIZE, IMAGE_SIZE)) for image in images_rgb]
        
        # add random rotate
        image_rotated = [rotate(image, np.random.random_sample()*60-30) for image in image_resized]

        # add random gamma adjust
        image_gamma_adjusted = [adjust_gamma(image, getRndGamma()) for image in image_rotated]

        self.images = [rgb2gray(item) for item in image_gamma_adjusted]
        
        # convert list to numpy array
        self.images = array(self.images)
        self.labels = array(self.labels)

# Specify feature
feature_columns = [tf.feature_column.numeric_column("x", shape=[IMAGE_SIZE, IMAGE_SIZE])]

# Build 8 layer DNN classifier
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns, # The input features to our model
    hidden_units=[4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096], # 8 hidden layers
    n_classes=CHARSET_SIZE, # survived or not {1, 0}
    model_dir="../dfs/checkpoint/dnn10_model_b", # Path to where checkpoints etc are stored
    optimizer=tf.train.RMSPropOptimizer(
        learning_rate=0.00001),
    dropout=0.1)

for i in range(5):
    train_data = DataSetLoader(data_dir='../data/train_/')
    print ('Train data loaded for the %d times.'%(i+1))
    # Define the training inputs
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": input(train_data)[0]},
        y=input(train_data)[1],
        num_epochs=None,
        batch_size=50,
        shuffle=True
    )

    classifier.train(input_fn=train_input_fn, steps=10000)

print ('Train done, begin to test ...')
test_data = DataSetLoader(data_dir='../data/test_/')
# Define the test inputs
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input(test_data)[0]},
    y=input(test_data)[1],
    num_epochs=1,
    shuffle=False
)

# Evaluate accuracy
accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]
print("\nTest Accuracy: {0:f}%\n".format(accuracy_score*100))

下面是单独测试：

In [ ]:
# from: https://www.kaggle.com/jeffcarp/example-save-and-load-a-tensorflow-model

import tensorflow as tf
import numpy as np
import os
import random
from numpy import array

from skimage.io import imread, imsave
from skimage.color import rgb2gray
from skimage.transform import resize

from tensorflow.contrib.learn.python.learn.estimators import run_config
from tensorflow.contrib.training.python.training import hparam

IMAGE_SIZE = 64
# CHARSET_SIZE = 3755
CHARSET_SIZE = 37

def input(dataset):
    return dataset.images, dataset.labels

class DataSetLoader:
    def __init__(self, data_dir):
        # Set CHARSET_SIZE to a small value if available computation power is limited.
        truncate_path = data_dir + ('%05d' % CHARSET_SIZE)
        print('Now processing path: ', truncate_path)
        image_names = []
        for root, sub_folder, file_list in os.walk(data_dir):
            if root < truncate_path:
                image_names += [os.path.join(root, file_path) for file_path in file_list]
        random.shuffle(image_names)
        self.labels = [int(file_name[len(data_dir):].split(os.sep)[0]) for file_name in image_names]
        images_rgb = [imread(file_name) for file_name in image_names]
        image_resized = [resize(image, (IMAGE_SIZE, IMAGE_SIZE)) for image in images_rgb]
        self.images = [rgb2gray(item) for item in image_resized]
        
        # convert list to numpy array
        self.images = array(self.images)
        self.labels = array(self.labels)

# Specify feature
feature_columns = [tf.feature_column.numeric_column("x", shape=[IMAGE_SIZE, IMAGE_SIZE])]

def make_estimator(model_dir):
    config = run_config.RunConfig(model_dir=model_dir)

    return tf.estimator.DNNClassifier (
        config=config,
        feature_columns=feature_columns, # The input features to our model
    hidden_units=[4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096], # 8 hidden layers
    n_classes=CHARSET_SIZE, # survived or not {1, 0}
    optimizer=tf.train.RMSPropOptimizer(
        learning_rate=0.00001),
    dropout=0.1)
    

MODEL_DIR = "../dfs/checkpoint/dnn10_model_b"
model_from_checkpoint = make_estimator(MODEL_DIR)

test_data = DataSetLoader(data_dir='../data/test_/')
# Define the test inputs
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input(test_data)[0]},
    y=input(test_data)[1],
    num_epochs=1,
    shuffle=False
)

predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input(test_data)[0]},
    num_epochs=1,
    shuffle=False)

# Evaluate accuracy
accuracy_score = model_from_checkpoint.evaluate(input_fn=test_input_fn)["accuracy"]
print("\nTest Accuracy: {0:f}%\n".format(accuracy_score*100))
